In [ ]:
# CIFAR-10 CNN 모델 - TensorFlow 초보자용 완전한 코드
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# # GPU 메모리 증가 방식 설정 (선택사항)
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         print(e)

# print(f"TensorFlow 버전: {tf.__version__}")
# print(f"사용 가능한 GPU: {len(gpus)}개")

# 1. 하이퍼파라미터 설정
BATCH_SIZE = 32        # 한 번에 처리할 이미지 개수
LEARNING_RATE = 0.001  # 학습률
EPOCHS = 10           # 전체 데이터를 몇 번 반복할지
IMG_HEIGHT = 32       # CIFAR-10 이미지 높이
IMG_WIDTH = 32        # CIFAR-10 이미지 너비
NUM_CLASSES = 10      # 클래스 개수

# 2. 데이터셋 로드 및 전처리
print("CIFAR-10 데이터셋 로드 중...")
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# 데이터 정보 출력
print(f"훈련 데이터 shape: {x_train.shape}")
print(f"훈련 레이블 shape: {y_train.shape}")
print(f"테스트 데이터 shape: {x_test.shape}")
print(f"테스트 레이블 shape: {y_test.shape}")

# CIFAR-10 클래스 이름
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# 3. 데이터 전처리
# 픽셀 값을 0-1 범위로 정규화
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 레이블을 원-핫 인코딩으로 변환
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

print("데이터 전처리 완료!")
print(f"정규화 후 픽셀 범위: {x_train.min():.1f} ~ {x_train.max():.1f}")
print(f"원-핫 인코딩 후 레이블 shape: {y_train.shape}")

# 4. 샘플 이미지 시각화
plt.figure(figsize=(12, 8))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.imshow(x_train[i])
    plt.title(f'{class_names[np.argmax(y_train[i])]}')
    plt.axis('off')
plt.suptitle('CIFAR-10 샘플 이미지', fontsize=16)
plt.tight_layout()
plt.show()

# 5. CNN 모델 구성 (Sequential API 사용)
print("CNN 모델 구성 중...")

# Sequential 모델 생성
model = keras.Sequential()

# 첫 번째 컨볼루션 블록: 32x32x3 → 32x32x32 → 16x16x32
model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 두 번째 컨볼루션 블록: 16x16x32 → 16x16x64 → 8x8x64
model.add(layers.Conv2D(64, (3, 3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 세 번째 컨볼루션 블록: 8x8x64 → 8x8x128 → 4x4x128
model.add(layers.Conv2D(128, (3, 3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# 완전 연결 층을 위해 1차원으로 펼치기
model.add(layers.Flatten())

# 완전 연결 층 (분류기)
model.add(layers.Dense(256))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))  # 과적합 방지

# 출력 층 (10개 클래스)
model.add(layers.Dense(NUM_CLASSES))
model.add(layers.Activation('softmax'))

# 6. 모델 컴파일
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',  # 다중 클래스 분류용
    metrics=['accuracy']
)

# 모델 구조 출력
print("모델 구조:")
model.summary()

# 7. 데이터 증강 (선택사항)
# 훈련 성능 향상을 위한 데이터 증강
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,        # 10도 범위에서 랜덤 회전
    width_shift_range=0.1,    # 수평으로 10% 이동
    height_shift_range=0.1,   # 수직으로 10% 이동
    horizontal_flip=True,     # 수평 뒤집기
    zoom_range=0.1           # 10% 확대/축소
)

# 훈련 데이터에 데이터 증강 적용
datagen.fit(x_train)

# 8. 콜백 함수 설정
# 훈련 과정에서 최고 성능 모델 저장
checkpoint = keras.callbacks.ModelCheckpoint(
    'best_cifar10_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# 학습률 감소 (성능 개선이 없으면 학습률 감소)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

# 조기 종료 (과적합 방지)
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True
)

# 9. 모델 훈련
print("모델 훈련 시작!")

# 데이터 증강을 사용한 훈련
# history = model.fit(
#     datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
#     steps_per_epoch=len(x_train) // BATCH_SIZE,
#     epochs=EPOCHS,
#     validation_data=(x_test, y_test),
#     callbacks=[checkpoint, reduce_lr, early_stop],
#     verbose=1
# )

# 데이터 증강 없이 기본 훈련을 원할 경우:
history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(x_test, y_test),
    callbacks=[checkpoint, reduce_lr, early_stop],
    verbose=1
)

# 10. 훈련 결과 시각화
def plot_training_history(history):
    """훈련 과정 시각화"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # 정확도 그래프
    ax1.plot(history.history['accuracy'], label='훈련 정확도')
    ax1.plot(history.history['val_accuracy'], label='검증 정확도')
    ax1.set_title('모델 정확도')
    ax1.set_ylabel('정확도')
    ax1.set_xlabel('에포크')
    ax1.legend()
    ax1.grid(True)

    # 손실 그래프
    ax2.plot(history.history['loss'], label='훈련 손실')
    ax2.plot(history.history['val_loss'], label='검증 손실')
    ax2.set_title('모델 손실')
    ax2.set_ylabel('손실')
    ax2.set_xlabel('에포크')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

plot_training_history(history)

# 11. 모델 평가
print("모델 평가 중...")
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"테스트 손실: {test_loss:.4f}")
print(f"테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# 12. 예측 결과 시각화
def show_predictions(model, x_test, y_test, class_names, num_images=12):
    """예측 결과 시각화"""
    # 랜덤하게 이미지 선택
    indices = np.random.choice(len(x_test), num_images, replace=False)

    # 예측 수행
    predictions = model.predict(x_test[indices])
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test[indices], axis=1)

    # 시각화
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        plt.subplot(3, 4, i + 1)
        plt.imshow(x_test[indices[i]])

        # 예측이 맞으면 파란색, 틀리면 빨간색
        color = 'blue' if predicted_classes[i] == true_classes[i] else 'red'

        plt.title(f'실제: {class_names[true_classes[i]]}\n'
                 f'예측: {class_names[predicted_classes[i]]}\n'
                 f'신뢰도: {np.max(predictions[i]):.2f}',
                 color=color, fontsize=10)
        plt.axis('off')

    plt.suptitle('예측 결과 (파란색: 정답, 빨간색: 오답)', fontsize=16)
    plt.tight_layout()
    plt.show()

print("예측 결과 시각화:")
show_predictions(model, x_test, y_test, class_names)

# 13. 클래스별 성능 분석
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# 전체 테스트 데이터에 대한 예측
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# 분류 보고서 출력
print("\n클래스별 성능 분석:")
print(classification_report(y_true_classes, y_pred_classes,
                          target_names=class_names))

# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('혼동 행렬 (Confusion Matrix)')
plt.ylabel('실제 클래스')
plt.xlabel('예측 클래스')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()
plt.show()

# 14. 모델 저장
model.save('cifar10_cnn_final_model.h5')
print("모델이 'cifar10_cnn_final_model.h5'로 저장되었습니다.")

# 15. 훈련 완료 요약
print("\n=== 훈련 완료 요약 ===")
print(f"최고 검증 정확도: {max(history.history['val_accuracy']):.4f}")
print(f"최종 테스트 정확도: {test_accuracy:.4f}")
print(f"총 훈련 에포크: {len(history.history['loss'])}")
print(f"모델 파라미터 수: {model.count_params():,}")

# 16. 추가 분석: 잘못 분류된 이미지들
def analyze_misclassified(model, x_test, y_test, class_names, num_images=8):
    """잘못 분류된 이미지 분석"""
    predictions = model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)

    # 잘못 분류된 이미지 찾기
    misclassified_indices = np.where(predicted_classes != true_classes)[0]

    # 랜덤하게 선택
    selected_indices = np.random.choice(misclassified_indices,
                                      min(num_images, len(misclassified_indices)),
                                      replace=False)

    plt.figure(figsize=(15, 10))
    for i, idx in enumerate(selected_indices):
        plt.subplot(2, 4, i + 1)
        plt.imshow(x_test[idx])
        plt.title(f'실제: {class_names[true_classes[idx]]}\n'
                 f'예측: {class_names[predicted_classes[idx]]}\n'
                 f'신뢰도: {np.max(predictions[idx]):.2f}',
                 color='red', fontsize=10)
        plt.axis('off')

    plt.suptitle('잘못 분류된 이미지들', fontsize=16)
    plt.tight_layout()
    plt.show()

print("\n잘못 분류된 이미지 분석:")
analyze_misclassified(model, x_test, y_test, class_names)